In [4]:
import locale
import pandas as pd
import numpy as np
from scipy import stats

def formatear_moneda(valor):
    """
    Formatea un valor numérico como una cadena de texto con formato monetario.

    :param valor: El valor numérico a formatear.
    :param locale_str: La configuración regional a usar para el formateo.
    :return: Una cadena de texto con el valor formateado como moneda.
    """
    locale.setlocale(locale.LC_ALL, '')
    return locale.currency(valor, grouping=True)


def compute_stats(column, alpha = 0.95, formatter=lambda x:x):
    mean = column.mean()
    min = column.min()
    max = column.max()
    std = column.std()

    n = len(column)
    stderr = std / np.sqrt(n)
    h = stderr * stats.t.ppf((1 + alpha) / 2, n - 1)
    bottom_top_interval = mean - h
    top_interval = mean + h

    return {
        "desviación estándar": formatter(std),
        "mínimo": formatter(min),
        "mínimo del intervalo de confianza": formatter(bottom_top_interval),
        "media": formatter(mean),
        "máximo del intervalo de confianza": formatter(top_interval),
        "máximo": formatter(max)       
    }


In [5]:
df = pd.read_csv('clients.csv')
df.head()

,Unnamed: 0,total de clientes fugados,total de clientes fugados en la mañana,total de clientes fugados al medio día,total de clientes fugados en la tarde,total de ingresos,total de clientes de tipo A fugados,Tiempo promedio en el sistema clientes tipo A,Tiempo promedio en la sala de espera clientes tipo A,total de clientes de tipo B fugados,...,Tiempo promedio en el sistema clientes tipo C,Tiempo promedio en la sala de espera clientes tipo C,Porcentaje de tiempo de trabajo el modulo 0.0 empleado en la atención de clientes,Porciento de clientes atendidos por el modulo 0.0,Porcentaje de tiempo de trabajo el modulo 1.0 empleado en la atención de clientes,Porciento de clientes atendidos por el modulo 1.0,Porcentaje de tiempo de trabajo el modulo 2.0 empleado en la atención de clientes,Porciento de clientes atendidos por el modulo 2.0,Porcentaje de tiempo de trabajo el modulo 3.0 empleado en la atención de clientes,Porciento de clientes atendidos por el modulo 3.0
0,0,17.579909,10.447761,39.814815,10.204082,6.708002e+06,24.107143,57.362557,54.551017,15.044248,...,31.795851,25.272860,93.493553,23.059361,102.084282,24.885845,61.222715,15.296804,89.475907,19.178082
1,1,13.053097,0.719424,28.695652,12.626263,9.763594e+06,10.638298,30.545250,28.184794,4.587156,...,59.456874,60.761974,101.239480,19.911504,94.087161,30.973451,85.778641,17.256637,69.291608,18.805310
2,2,19.742489,8.108108,33.928571,20.388350,7.222965e+06,17.446809,43.845459,41.236280,10.619469,...,91.690034,94.331259,106.687478,18.025751,100.181168,24.248927,102.057976,17.167382,84.295031,20.815451
3,3,27.868852,0.000000,57.615894,24.500000,-2.291536e+06,27.490040,50.159160,41.984789,22.689076,...,50.824499,42.239525,98.360743,17.008197,96.647137,21.106557,86.027846,15.368852,85.019016,18.647541
4,4,19.361277,9.142857,48.760331,10.731707,9.292008e+06,28.225806,50.949478,50.803142,6.521739,...,39.888984,38.482689,100.548223,19.760479,102.243651,22.355289,94.971549,18.562874,100.475382,19.960080


In [10]:
df.columns

Index(['Unnamed: 0', 'total de clientes fugados',
       'total de clientes fugados en la mañana',
       'total de clientes fugados al medio día',
       'total de clientes fugados en la tarde', 'total de ingresos',
       'total de clientes de tipo A fugados',
       'Tiempo promedio en el sistema clientes tipo A',
       'Tiempo promedio en la sala de espera clientes tipo A',
       'total de clientes de tipo B fugados',
       'Tiempo promedio en el sistema clientes tipo B',
       'Tiempo promedio en la sala de espera clientes tipo B',
       'total de clientes de tipo C fugados',
       'Tiempo promedio en el sistema clientes tipo C',
       'Tiempo promedio en la sala de espera clientes tipo C',
       'Porcentaje de tiempo de trabajo el modulo 0.0 empleado en la atención de clientes',
       'Porciento de clientes atendidos por el modulo 0.0',
       'Porcentaje de tiempo de trabajo el modulo 1.0 empleado en la atención de clientes',
       'Porciento de clientes atendidos por 

In [8]:
pd.DataFrame({
    "total de clientes fugados": compute_stats(df["total de clientes fugados"]),
    "total de clientes de tipo A fugados":  compute_stats(df["total de clientes de tipo A fugados"]),
    "total de clientes de tipo B fugados":  compute_stats(df["total de clientes de tipo B fugados"]),
    "total de clientes de tipo C fugados":  compute_stats(df["total de clientes de tipo C fugados"]),
    "total de clientes fugados en la mañana": compute_stats(df["total de clientes fugados en la mañana"]),
    "total de clientes fugados al medio día": compute_stats(df["total de clientes fugados al medio día"]),
    "total de clientes fugados en la tarde": compute_stats(df["total de clientes fugados en la tarde"]),

}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes fugados,3.967443,3.738318,20.288125,20.300874,20.313624,40.037244
total de clientes de tipo A fugados,5.506940,2.727273,22.233534,22.251231,22.268928,50.793651
total de clientes de tipo B fugados,4.305148,0.000000,13.447444,13.461278,13.475113,41.085271
total de clientes de tipo C fugados,6.720359,1.111111,23.502812,23.524407,23.546003,58.064516
total de clientes fugados en la mañana,4.102700,0.000000,5.096901,5.110085,5.123269,39.080460
total de clientes fugados al medio día,8.383809,5.813953,41.658719,41.685660,41.712601,78.985507
total de clientes fugados en la tarde,5.599601,0.518135,17.300351,17.318346,17.336340,48.117155


In [9]:
pd.DataFrame({
    "total de ingresos": compute_stats(df["total de ingresos"], formatter=formatear_moneda),
}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de ingresos,"$3,329,018.53","-$16,147,860.04","$5,436,942.94","$5,447,640.74","$5,458,338.53","$18,821,653.54"


In [11]:
# Estadisticas de tiempo en el sistema

pd.DataFrame({
    "Tiempo promedio en el sistema clientes tipo A": compute_stats(df["Tiempo promedio en el sistema clientes tipo A"]),
    "Tiempo promedio en el sistema clientes tipo B": compute_stats(df["Tiempo promedio en el sistema clientes tipo B"]),
    "Tiempo promedio en el sistema clientes tipo C": compute_stats(df["Tiempo promedio en el sistema clientes tipo C"]),
}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en el sistema clientes tipo A,8.878611,22.185996,49.782238,49.810769,49.839301,92.818209
Tiempo promedio en el sistema clientes tipo B,4.120774,19.369066,34.177684,34.190926,34.204168,65.964771
Tiempo promedio en el sistema clientes tipo C,13.065954,18.186221,52.509567,52.551555,52.593542,122.704963


In [12]:
# Estadisticas de tiempo en la lista de espera

pd.DataFrame({
    "Tiempo promedio en la sala de espera clientes tipo A": compute_stats(df["Tiempo promedio en la sala de espera clientes tipo A"]),
    "Tiempo promedio en la sala de espera clientes tipo B": compute_stats(df["Tiempo promedio en la sala de espera clientes tipo B"]),
    "Tiempo promedio en la sala de espera clientes tipo C": compute_stats(df["Tiempo promedio en la sala de espera clientes tipo C"]),
}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en la sala de espera clientes tipo A,9.261358,18.939348,46.954203,46.983965,47.013726,94.531363
Tiempo promedio en la sala de espera clientes tipo B,3.054713,17.458579,28.913325,28.923141,28.932957,47.860149
Tiempo promedio en la sala de espera clientes tipo C,14.566539,12.575319,49.573979,49.620788,49.667598,132.217400


In [14]:
pd.DataFrame({
    "Porcentaje de tiempo de trabajo el modulo 0.0 empleado en la atención de clientes": compute_stats(df["Porcentaje de tiempo de trabajo el modulo 0.0 empleado en la atención de clientes"]),
    "Porcentaje de tiempo de trabajo el modulo 1.0 empleado en la atención de clientes": compute_stats(df["Porcentaje de tiempo de trabajo el modulo 1.0 empleado en la atención de clientes"]),
    "Porcentaje de tiempo de trabajo el modulo 2.0 empleado en la atención de clientes": compute_stats(df["Porcentaje de tiempo de trabajo el modulo 2.0 empleado en la atención de clientes"]),
    "Porcentaje de tiempo de trabajo el modulo 3.0 empleado en la atención de clientes": compute_stats(df["Porcentaje de tiempo de trabajo el modulo 3.0 empleado en la atención de clientes"]),
}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Porcentaje de tiempo de trabajo el modulo 0.0 empleado en la atención de clientes,4.913668,68.877484,98.801133,98.816923,98.832713,122.048365
Porcentaje de tiempo de trabajo el modulo 1.0 empleado en la atención de clientes,3.471459,79.044860,101.192708,101.203864,101.215019,122.940089
Porcentaje de tiempo de trabajo el modulo 2.0 empleado en la atención de clientes,9.792630,26.847030,82.490905,82.522374,82.553843,118.472054
Porcentaje de tiempo de trabajo el modulo 3.0 empleado en la atención de clientes,7.575442,49.486761,92.432437,92.456781,92.481124,118.008181


In [15]:
pd.DataFrame({
    "Porciento de clientes atendidos por el modulo 0.0": compute_stats(df["Porciento de clientes atendidos por el modulo 0.0"]),
    "Porciento de clientes atendidos por el modulo 1.0": compute_stats(df["Porciento de clientes atendidos por el modulo 1.0"]),
    "Porciento de clientes atendidos por el modulo 2.0": compute_stats(df["Porciento de clientes atendidos por el modulo 2.0"]),
    "Porciento de clientes atendidos por el modulo 3.0": compute_stats(df["Porciento de clientes atendidos por el modulo 3.0"]),
}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Porciento de clientes atendidos por el modulo 0.0,1.882515,11.567164,19.467346,19.473396,19.479445,28.705882
Porciento de clientes atendidos por el modulo 1.0,2.287713,13.137255,22.994861,23.002212,23.009564,33.262260
Porciento de clientes atendidos por el modulo 2.0,1.928022,6.320542,16.246372,16.252568,16.258763,24.832215
Porciento de clientes atendidos por el modulo 3.0,2.121750,11.673152,20.964132,20.970950,20.977768,30.905077
